##From Milestone 2 Notebook

In [ ]:
import torch, torchtext, numpy as np
import pandas as pd, csv
from torch import nn, optim
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import pdb
torch.manual_seed(291)
np.random.seed(291)
import pandas as pd

In [ ]:
#Code adapted and inspired by movie recommender model in lecture 6
class Dataset(torch.utils.data.Dataset):
    def __init__(self, fn, books):
        self.dataframe = pd.read_csv(fn)
        self.books = pd.read_csv(books)
        u2n = { u: n for n, u in enumerate(self.dataframe['user_id'].unique()) } 
        m2n = { m: n for n, m in enumerate(self.dataframe['book_id'].unique()) } 
        self.dataframe['user_id'] = self.dataframe['user_id'].apply(lambda u: u2n[u])
        self.dataframe['book_id'] = self.dataframe['book_id'].apply(lambda m: m2n[m])
        self.coords = torch.LongTensor(self.dataframe[['user_id','book_id']].values) # (userId,bookId) <- coordinates
        self.ratings = torch.FloatTensor(self.dataframe['rating'].values)
        self.n_users = self.dataframe['user_id'].nunique()
        self.n_books = self.dataframe['book_id'].nunique()

    def __len__(self):
        return len(self.coords)
    
    def get_book_title(self, book_id): #This method currently has an off by one error, waiting to address until we use this code for serving recommnedations for users, as datasets may change by then - Declan
      return (self.books[self.books["book_id"] == book_id]).loc[:,"original_title"].values[0] 

    def __getitem__(self, i):  
      return (self.coords[i], self.ratings[i])           

In [ ]:
!wget https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/ratings.csv
!wget https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/books.csv
ds_full = Dataset('ratings.csv', 'books.csv')
n_train = int(0.72 * len(ds_full))
n_test = int(0.18 * len(ds_full))
n_RL = len(ds_full) - n_train - n_test
rng = torch.Generator().manual_seed(291)
ds_train, ds_test, ds_RL = torch.utils.data.random_split(ds_full, [n_train, n_test, n_RL], rng)
len(ds_full)

--2021-04-12 00:08:53--  https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/ratings.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 72126826 (69M) [text/plain]
Saving to: ‘ratings.csv.1’

ratings.csv.1       100%[===================>]  68.79M   150MB/s    in 0.5s    

2021-04-12 00:08:53 (150 MB/s) - ‘ratings.csv.1’ saved [72126826/72126826]

--2021-04-12 00:08:54--  https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/books.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3286659 (3.1M) [text/plain]

5976479

In [ ]:
type(ds_train)

torch.utils.data.dataset.Subset

In [ ]:
#Code adapted and inspired by movie recommender model in lecture 6
class BookRecommenderEmbeddingML(nn.Module):
  def __init__(self, n_users, n_books, emb_dim):
    super(BookRecommenderEmbeddingML, self).__init__()
    self.user_embedding = nn.Embedding(n_users, emb_dim)
    self.user_bias = nn.Embedding(n_users, 1) #I question if this is helpful for our use of this model, due to our plan to use users in this dataset to represent multiple users - Declan
    self.book_embedding = nn.Embedding(n_books, emb_dim)
    self.book_bias = nn.Embedding(n_books, 1)
    nn.init.xavier_uniform_(self.user_embedding.weight)
    nn.init.xavier_uniform_(self.book_embedding.weight)
    nn.init.zeros_(self.user_bias.weight)
    nn.init.zeros_(self.book_bias.weight)
    self.dropout = nn.Dropout(0.25)
  
  def forward(self, samples):
    users = self.user_embedding(samples[:,0]) # gets embedding of users
    users = self.dropout(users)
    books = self.book_embedding(samples[:,1])
    books = self.dropout(books)
    dot = (users * books).sum(1)
    user_b = self.user_bias(samples[:,0]).squeeze()
    book_b = self.book_bias(samples[:,1]).squeeze()
    return torch.sigmoid(dot + user_b + book_b) * 5.5

In [ ]:
device = torch.device('cuda:0')

def run_test(model, ldr, crit):
    total_loss, total_count = 0, 0
    model.eval()
    tq_iters = tqdm(ldr, leave=False, desc='test iter')
    with torch.no_grad():
        for coords, labels in tq_iters:
            coords, labels = coords.to(device), labels.to(device)
            preds = model(coords)
            loss = crit(preds, labels)
            total_loss += loss.item() * labels.size(0)
            total_count += labels.size(0)
            tq_iters.set_postfix({'loss': total_loss/total_count}, refresh=True)
    return total_loss / total_count

def run_train(model, ldr, crit, opt, sched):
    model.train()
    total_loss, total_count = 0, 0
    tq_iters = tqdm(ldr, leave=False, desc='train iter')
    for (coords, labels) in tq_iters:
        opt.zero_grad()
        coords, labels = coords.to(device), labels.to(device)
        preds = model(coords)
        loss = crit(preds, labels)
        loss.backward()
        opt.step()
        sched.step()
        total_loss += loss.item() * labels.size(0)
        total_count += labels.size(0)
        tq_iters.set_postfix({'loss': total_loss/total_count}, refresh=True)
    return total_loss / total_count

def run_all(model, ldr_train, ldr_test, crit, opt, sched, n_epochs=10):
    best_loss = np.inf
    tq_epochs = tqdm(range(n_epochs), desc='epochs', unit='ep')
    for epoch in tq_epochs:
        train_loss = run_train(model, ldr_train, crit, opt, sched)
        test_loss = run_test(model, ldr_test, crit)
        tqdm.write(f'epoch {epoch}   train loss {train_loss:.6f}    test loss {test_loss:.6f}')
        if test_loss < best_loss:
            best_loss = test_loss
            tq_epochs.set_postfix({'bE': epoch, 'bL': best_loss}, refresh=True)

In [ ]:
model = BookRecommenderEmbeddingML(ds_full.n_users, ds_full.n_books, 24) 
model.to(device)

ldr_train = torch.utils.data.DataLoader(ds_train, batch_size=32, shuffle=True)
ldr_test = torch.utils.data.DataLoader(ds_test, batch_size=32)

n_epochs = 2

crit = nn.MSELoss().to(device)
opt = optim.SGD(model.parameters(), lr=1e-6, momentum=0.9)
sched = optim.lr_scheduler.OneCycleLR(opt, max_lr=0.1, steps_per_epoch=len(ldr_train), epochs=n_epochs)

run_all(model, ldr_train, ldr_test, crit, opt, sched, n_epochs)

epoch 0   train loss 0.869606    test loss 0.750823


epoch 1   train loss 0.721712    test loss 0.707309



In [ ]:
def parameter_test(emb, batch, n_epochs, learn, p, max_learn, input_model, new_model=True):
  if new_model:
    model = BookRecommenderEmbeddingML(ds_full.n_users, ds_full.n_books, emb)
  else:
    model = input_model
  model.to(device)
  ldr_train = torch.utils.data.DataLoader(ds_train, batch_size=batch, shuffle=True)
  ldr_test = torch.utils.data.DataLoader(ds_test, batch_size=batch)

  crit = nn.MSELoss().to(device)
  opt = optim.SGD(model.parameters(), lr=learn,momentum=p)
  sched = optim.lr_scheduler.OneCycleLR(opt, max_lr=max_learn, steps_per_epoch=len(ldr_train), epochs= n_epochs)

  run_all(model, ldr_train, ldr_test, crit, opt, sched, n_epochs)
  
  return model


In [ ]:
#Note this code was ran before adding the ability to reuse a model in a a parameter test
print("Testing embedding dimensions")
print("Embedding dimensions: 24")
parameter_test(24,32,10,1e-6,0.9,0.1)
print("Embedding dimensions: 36")
parameter_test(36,32,10,1e-6,0.9,0.1)
print("Embedding dimnesions: 16")
parameter_test(16,32,10,1e-6,0.9,0.1)

Testing embedding dimensions
Embedding dimensions: 24


epoch 0   train loss 0.929395    test loss 0.755043


epoch 1   train loss 0.744440    test loss 0.739025


epoch 2   train loss 0.710669    test loss 0.711210


epoch 3   train loss 0.637788    test loss 0.708742


...

epoch 5   train loss 0.569972    test loss 0.719912


epoch 6   train loss 0.537119    test loss 0.724191


epoch 7   train loss 0.502902    test loss 0.728200


epoch 8   train loss 0.465979    test loss 0.733057


epoch 9   train loss 0.434134    test loss 0.736313



In [ ]:
print("Testing dropout")
parameter_test(16,32,10,1e-6,0.9,0.1)

Testing dropout


epoch 0   train loss 0.929140    test loss 0.754891


epoch 1   train loss 0.745515    test loss 0.741954


epoch 2   train loss 0.729752    test loss 0.724180


epoch 3   train loss 0.707502    test loss 0.714512


...

epoch 5   train loss 0.673317    test loss 0.702310


epoch 6   train loss 0.653071    test loss 0.694003


epoch 7   train loss 0.627640    test loss 0.681285


epoch 8   train loss 0.598085    test loss 0.670720


epoch 9   train loss 0.572408    test loss 0.668701



In [ ]:
print("Testing dropout over a longer period of time:")
long_time_dropout = parameter_test(16,32,25,1e-6,0.9,0.1,0,new_model=True)

Testing dropout over a longer period of time:


epoch 0   train loss 0.954837    test loss 0.779787


epoch 1   train loss 0.747807    test loss 0.740060


epoch 2   train loss 0.731545    test loss 0.729834


epoch 3   train loss 0.716316    test loss 0.716036


...

epoch 20   train loss 0.585806    test loss 0.677626


epoch 21   train loss 0.572749    test loss 0.673526


epoch 22   train loss 0.560809    test loss 0.670502


epoch 23   train loss 0.550914    test loss 0.669363


epoch 24   train loss 0.545088    test loss 0.669209



##New Training

In [ ]:
#Further model testing
model = parameter_test(16,32,50,1e-6,0.9,0.1,0,new_model=True)

epoch 0   train loss 0.962094    test loss 0.793157


epoch 1   train loss 0.756603    test loss 0.743764


epoch 2   train loss 0.730384    test loss 0.733749


epoch 3   train loss 0.720450    test loss 0.722244


epoch 4   train loss 0.704965    test loss 0.709545


...

epoch 45   train loss 0.549915    test loss 0.670573


epoch 46   train loss 0.545171    test loss 0.669561


epoch 47   train loss 0.541355    test loss 0.669087


epoch 48   train loss 0.538419    test loss 0.668836


epoch 49   train loss 0.537064    test loss 0.668821



In [ ]:
print("Further parameter testing / different embedding and different momentum")
model = BookRecommenderEmbeddingML(ds_full.n_users, ds_full.n_books, 24)
model.to(device)
ldr_train = torch.utils.data.DataLoader(ds_train, batch_size=32, shuffle=True)
ldr_test = torch.utils.data.DataLoader(ds_test, batch_size=32)

crit = nn.MSELoss().to(device)
opt = optim.SGD(model.parameters(), lr=1e-6,momentum=0.75)
sched = optim.lr_scheduler.OneCycleLR(opt, max_lr=0.1, steps_per_epoch=len(ldr_train), epochs= 10)

run_all(model, ldr_train, ldr_test, crit, opt, sched, 10)

Further parameter testing / different embedding and different momentum


epoch 0   train loss 0.943825    test loss 0.759282


epoch 1   train loss 0.746797    test loss 0.744597


epoch 2   train loss 0.728670    test loss 0.725827


epoch 3   train loss 0.698367    test loss 0.718664


...

epoch 5   train loss 0.648449    test loss 0.711316


epoch 6   train loss 0.622411    test loss 0.704270


epoch 7   train loss 0.589487    test loss 0.694578


epoch 8   train loss 0.553941    test loss 0.686430


epoch 9   train loss 0.523856    test loss 0.684858



In [ ]:
print("Similar to above, increased dropout")
model0 = BookRecommenderEmbeddingML(ds_full.n_users, ds_full.n_books, 24)
model0.dropout = nn.Dropout(0.35)
model0.to(device)
ldr_train = torch.utils.data.DataLoader(ds_train, batch_size=32, shuffle=True)
ldr_test = torch.utils.data.DataLoader(ds_test, batch_size=32)

crit = nn.MSELoss().to(device)
opt = optim.SGD(model0.parameters(), lr=1e-6,momentum=0.75)
sched = optim.lr_scheduler.OneCycleLR(opt, max_lr=0.1, steps_per_epoch=len(ldr_train), epochs= 15)

run_all(model0, ldr_train, ldr_test, crit, opt, sched, 15)

Similar to above, increased dropout


epoch 0   train loss 0.959034    test loss 0.771281


epoch 1   train loss 0.746894    test loss 0.744177


epoch 2   train loss 0.734556    test loss 0.734601


epoch 3   train loss 0.723070    test loss 0.725995


...

epoch 10   train loss 0.646188    test loss 0.700629


epoch 11   train loss 0.622885    test loss 0.690786


epoch 12   train loss 0.597495    test loss 0.682034


epoch 13   train loss 0.574053    test loss 0.676790


epoch 14   train loss 0.558994    test loss 0.675903



In [ ]:
print("Returning to 0.25 dropout, maintaing momentum of above, trying more embedding layers")
model1 = BookRecommenderEmbeddingML(ds_full.n_users, ds_full.n_books, 32)
#model0.dropout = nn.Dropout(0.35)
model1.to(device)
ldr_train = torch.utils.data.DataLoader(ds_train, batch_size=32, shuffle=True)
ldr_test = torch.utils.data.DataLoader(ds_test, batch_size=32)

crit = nn.MSELoss().to(device)
opt = optim.SGD(model1.parameters(), lr=1e-6,momentum=0.75)
sched = optim.lr_scheduler.OneCycleLR(opt, max_lr=0.1, steps_per_epoch=len(ldr_train), epochs= 15)

run_all(model1, ldr_train, ldr_test, crit, opt, sched, 15)

Returning to 0.25 dropout, maintaing momentum of above, trying more embedding layers


epoch 0   train loss 0.958904    test loss 0.771211


epoch 1   train loss 0.746711    test loss 0.744521


epoch 2   train loss 0.729058    test loss 0.726753


epoch 3   train loss 0.700458    test loss 0.717255


...

epoch 10   train loss 0.557782    test loss 0.718557


epoch 11   train loss 0.531928    test loss 0.710087


epoch 12   train loss 0.504024    test loss 0.703284


epoch 13   train loss 0.478925    test loss 0.699478


epoch 14   train loss 0.463132    test loss 0.698618



In [ ]:
print("Maintaining dropout, testing lower momemntum, returning to 24 layers")
model2 = BookRecommenderEmbeddingML(ds_full.n_users, ds_full.n_books, 24)
#model0.dropout = nn.Dropout(0.35)
model2.to(device)
ldr_train = torch.utils.data.DataLoader(ds_train, batch_size=32, shuffle=True)
ldr_test = torch.utils.data.DataLoader(ds_test, batch_size=32)

crit = nn.MSELoss().to(device)
opt2 = optim.SGD(model2.parameters(), lr=1e-6,momentum=0.5)
sched2 = optim.lr_scheduler.OneCycleLR(opt2, max_lr=0.1, steps_per_epoch=len(ldr_train), epochs= 15)

run_all(model2, ldr_train, ldr_test, crit, opt2, sched2, 15)

Maintaining dropout, testing lower momemntum, returning to 24 layers


epoch 0   train loss 0.959123    test loss 0.771223


epoch 1   train loss 0.747076    test loss 0.745637


epoch 2   train loss 0.731291    test loss 0.729084


epoch 3   train loss 0.706991    test loss 0.718924


...

epoch 10   train loss 0.589263    test loss 0.709375


epoch 11   train loss 0.565742    test loss 0.701531


epoch 12   train loss 0.541128    test loss 0.694473


epoch 13   train loss 0.517798    test loss 0.690093


epoch 14   train loss 0.502375    test loss 0.689434



In [ ]:
print("Maintaining dropout, testing higher momemntum, returning to 24 layers")
model3 = BookRecommenderEmbeddingML(ds_full.n_users, ds_full.n_books, 24)
#model0.dropout = nn.Dropout(0.35)
model3.to(device)
ldr_train = torch.utils.data.DataLoader(ds_train, batch_size=32, shuffle=True)
ldr_test = torch.utils.data.DataLoader(ds_test, batch_size=32)

crit = nn.MSELoss().to(device)
opt3 = optim.SGD(model3.parameters(), lr=1e-6,momentum=0.66)
sched3 = optim.lr_scheduler.OneCycleLR(opt3, max_lr=0.1, steps_per_epoch=len(ldr_train), epochs= 15)

run_all(model3, ldr_train, ldr_test, crit, opt3, sched3, 15)

Maintaining dropout, testing higher momemntum, returning to 24 layers


epoch 0   train loss 0.958956    test loss 0.771255


epoch 1   train loss 0.746876    test loss 0.745171


epoch 2   train loss 0.731829    test loss 0.728868


epoch 3   train loss 0.706678    test loss 0.717255


...

epoch 10   train loss 0.588631    test loss 0.707409


epoch 11   train loss 0.565188    test loss 0.699983


epoch 12   train loss 0.540799    test loss 0.692804


epoch 13   train loss 0.517638    test loss 0.689018


epoch 14   train loss 0.502249    test loss 0.688279



In [ ]:
print("Maintaining dropout, testing higher momemntum, returning to 24 layers")
model4 = BookRecommenderEmbeddingML(ds_full.n_users, ds_full.n_books, 24)
#model0.dropout = nn.Dropout(0.35)
model4.to(device)
ldr_train = torch.utils.data.DataLoader(ds_train, batch_size=32, shuffle=True)
ldr_test = torch.utils.data.DataLoader(ds_test, batch_size=32)

crit = nn.MSELoss().to(device)
opt4 = optim.SGD(model4.parameters(), lr=1e-6,momentum=0.8)
sched4 = optim.lr_scheduler.OneCycleLR(opt4, max_lr=0.1, steps_per_epoch=len(ldr_train), epochs= 15)

run_all(model4, ldr_train, ldr_test, crit, opt4, sched4, 15)

Maintaining dropout, testing higher momemntum, returning to 24 layers


epoch 0   train loss 0.959114    test loss 0.771723


epoch 1   train loss 0.747134    test loss 0.744593


epoch 2   train loss 0.732470    test loss 0.728805


epoch 3   train loss 0.707909    test loss 0.720066


...

epoch 10   train loss 0.588871    test loss 0.709586


epoch 11   train loss 0.565514    test loss 0.701480


epoch 12   train loss 0.540911    test loss 0.694794


epoch 13   train loss 0.517407    test loss 0.690840


epoch 14   train loss 0.502239    test loss 0.690036



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
torch.save({
            'train': ds_train,
            'test': ds_test,
            'RL': ds_RL
            
            }, '/content/drive/My Drive/datasets.pt')

In [ ]:
torch.save({
            'best-model' : model0 
}, '/content/drive/My Drive/best-model-apr11.pt')

##Citations
Below are citations for the use of the datasets used in the code above  
(Citations are in the style specified by the dataset compiler)  

@article{goodbooks2017,
    author = {Zajac, Zygmunt},
    title = {Goodbooks-10k: a new dataset for book recommendations},
    year = {2017},
    publisher = {FastML},
    journal = {FastML},
    howpublished = {\url{http://fastml.com/goodbooks-10k}},
}

In [ ]:
matrix = np.empty((ds_full.n_users, ds_full.n_books), int)
for i in range(ds_full.n_users):
  for j in range(ds_full.n_books):
    sample = torch.tensor([[i,j]])
    sample = sample.to(device)
    matrix[i][j] = model(sample)
    

In [ ]:
print(matrix[0][0])

NameError: ignored

In [ ]:
matrix = np.empty((ds_full.n_users, ds_full.n_books), int)
for i in range(ds_full.n_users):
  currUserRatings = []
  for j in range(ds_full.n_books):
    sample = torch.tensor([[i,j]])
    sample = sample.to(device)
    currUserRating.append(model(sample))
  matrix[i] = currUserRatings.sort()

In [ ]:
ratings = [[book_id1, rating1], [book_id1, rating1], [book_id1, rating1]]

bestDiff = np.abs(matrix[0][ratings[0][0]] - ratings[0][1])
bestUser = 0
for i in range(ds_full.n_users):
  diff = 0
  for rating in ratings:
    diff = diff + np.abs(matrix[i][rating[0]] - rating[1])
  if diff < bestDiff:
    bestDiff = diff
    bestUser = i

In [ ]:

NumBooks = 5 # Constant to determine how many books we will reccommend each time
bookRatings = matrix[bestUser]
def reccommendBooks(self, NumBooks):
  toReccommend = []
  toReccommend.append(bookRatings[0:NumBooks])
  bookRatings = bookRatings[NumBooks:]
  return toReccommend


  
  

In [ ]:

# userNumber = bestUser
userNumber = 0 # 0 right now for testing
t = torch.LongTensor([0])
t = t.to(device)
result = model.user_embedding(t) # get feature vector for user 0
user_feature_vector = np.array(result[0].tolist()).flatten()
print(user_feature_vector)



[ 0.04047814 -0.00125407  0.0146815   0.09409566 -0.01209675  0.06589172
 -0.00066815 -0.05898933  0.02309715 -0.00367716 -0.01564793  0.03521119
  0.01121175 -0.03043495 -0.06708327  0.07649221  0.03411641 -0.08947678
 -0.06959349  0.03025613 -0.0232045   0.00532378 -0.07223671  0.02035745]
